In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.utils import  normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from matplotlib import pyplot as plt

In [2]:
image_directory='datasets/'

In [3]:
no_tumor_images=os.listdir(image_directory+ 'no/')
yes_tumor_images=os.listdir(image_directory+ 'yes/')
dataset=[]
label=[]

In [4]:
INPUT_SIZE=64

In [5]:
for i, image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+ 'no/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

for i, image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+ 'yes/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)


In [6]:
dataset=np.array(dataset)
label=np.array(label)

x_train, x_test, y_train, y_test =train_test_split(dataset,label, test_size=0.2)

In [7]:
x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

In [8]:
model=Sequential()

model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#Binary CrossEntropy= 1, sigmoid
#CrossEntropy= 1, sigmoid

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, verbose=1, epochs=10, validation_data=(x_test, y_test), shuffle=False)

model.save('BrainTumor10Epochs5vs.h5')

Epoch 1/10
150/150 [==============================] - 5s 31ms/step - loss: 0.5656 - accuracy: 0.7133 - val_loss: 0.4494 - val_accuracy: 0.7867
Epoch 2/10
150/150 [==============================] - 4s 27ms/step - loss: 0.4479 - accuracy: 0.8050 - val_loss: 0.3271 - val_accuracy: 0.8650
Epoch 3/10
150/150 [==============================] - 4s 26ms/step - loss: 0.3494 - accuracy: 0.8546 - val_loss: 0.2465 - val_accuracy: 0.8867
Epoch 4/10
150/150 [==============================] - 4s 24ms/step - loss: 0.2880 - accuracy: 0.8854 - val_loss: 0.2073 - val_accuracy: 0.9183
Epoch 5/10
150/150 [==============================] - 4s 24ms/step - loss: 0.2235 - accuracy: 0.9121 - val_loss: 0.1981 - val_accuracy: 0.9200
Epoch 6/10
150/150 [==============================] - 4s 24ms/step - loss: 0.1692 - accuracy: 0.9350 - val_loss: 0.1368 - val_accuracy: 0.9633
Epoch 7/10
150/150 [==============================] - 4s 24ms/step - loss: 0.1269 - accuracy: 0.9567 - val_loss: 0.1090 - val_accuracy: 0.9633

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 activation (Activation)     (None, 62, 62, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 29, 29, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                    

In [10]:
p_pred = model.predict(x_test)
p_pred = p_pred.flatten()

19/19 [==============================] - 0s 14ms/step


In [16]:
y_pred = np.where(p_pred > 0.5, 1, 0)

In [17]:
print(confusion_matrix(y_test, y_pred))

[[309   7]
 [  5 279]]


In [18]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn,fp,fn,tp

(309, 7, 5, 279)

In [19]:
matrix = classification_report(y_test, y_pred)
print('Classification Report : \n', matrix)

Classification Report : 
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       316
           1       0.98      0.98      0.98       284

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



In [20]:
score = accuracy_score(y_test,y_pred)
score

0.98

In [21]:
print('Model Accuracy is :', score)

Model Accuracy is : 0.98
